In [2]:
search_for = 130
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.008218765258789062
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 749112551
type: [1, 1, 1, 1, 130] 130
cases of this type: 285610000
100000 49.760999511016145
200000 58.21508401244314
300000 69.79866800237167
400000 70.97799053835885
500000 67.45407833510085
600000 62.11639854268726
700000 70.57480235789271
800000 71.3363455108637
900000 61.90699117891138
1000000 61.85017297310812
1100000 42.78973116421097
1200000 33.24187785777813
1300000 59.653005297031214
1400000 56.181174527282295
1500000 56.517910760953335
1600000 58.92611965997495
1700000 59.838340772516105
1800000 60.89593860780752
1900000 55.66677126617551
2000000 61.78749013097479
2100000 56.51465132071664
2200000 48.68110053166385
2300000 30.14503315452824
2400000 54.60498585019565
2500000 60.370512987075905
2600000 53.70687969902277
2700000 59.42083399073979
2800000 58.526278998557586
2900000 57.65023058178285
3000000 54.392907903

29700000 19.278374400592476
29800000 55.68168438067857
29900000 55.32703654143936
30000000 59.21517411899909
30100000 61.21436371324532
30200000 63.36059213657649
30300000 57.09991812706575
30400000 66.8749927313284
30500000 60.40572869671953
30600000 23.409098580643516
30700000 66.12302878426131
30800000 61.688299921184054
30900000 66.07732722382434
31000000 41.8176866066329
31100000 42.62859048581818
31200000 60.96070903804276
31300000 62.86799847696183
31400000 58.91798178481862
31500000 60.63237181432227
31600000 57.12512352619547
31700000 60.90230317214041
31800000 51.238273282776994
31900000 18.87911990437856
32000000 57.51591012237422
32100000 53.77802642932736
32200000 61.05451703956099
32300000 54.40416113777819
32400000 64.34645580812568
32500000 57.23617107669039
32600000 63.16687923525345
32700000 52.50125249491927
32800000 30.91440173318859
32900000 63.618079307038194
33000000 60.818517967291186
33100000 71.03296488660727
33200000 53.91932451344452
33300000 37.376807976315

59800000 31.280016313774052
59900000 47.04895762170109
60000000 53.19629939481054
60100000 54.88304485210825
60200000 49.98274361511233
60300000 54.10319066513321
60400000 41.81723266288461
60500000 22.140205344506633
60600000 53.388872108151794
60700000 48.81695760960571
60800000 52.21056150462413
60900000 50.70604692188013
61000000 53.08752862542561
61100000 27.609606020579758
61200000 51.278339947260434
61300000 59.71865962269896
61400000 56.19433306073248
61500000 59.21266962858377
61600000 56.004330113099684
61700000 60.90623552094647
61800000 57.57402227105915
61900000 59.7306610695702
62000000 37.05447031633129
62100000 37.07282303178655
62200000 56.91477910423221
62300000 49.34263106377711
62400000 53.099966394604714
62500000 49.624265147000685
62600000 44.18248698095235
62700000 24.224362316314554
62800000 48.95389904221057
62900000 52.67701069466867
63000000 48.714042358511136
63100000 54.01723438239423
63200000 53.488425883135584
63300000 22.08874349677924
63400000 59.232679

89900000 48.78958913839851
90000000 52.47742496670151
90100000 48.98541421745705
90200000 55.81563820801677
90300000 53.832974659860255
90400000 55.75025809437023
90500000 52.4438052341088
90600000 53.47591500403269
90700000 47.96321739129935
90800000 24.506612089042697
90900000 44.71305406052581
91000000 48.70538352657733
91100000 49.510314686601774
91200000 25.431401122748827
91300000 38.10822333328336
91400000 51.61619680709507
91500000 47.739754378131515
91600000 30.841478611914933
91700000 41.90088749120621
91800000 46.14432765318226
91900000 53.76812672114099
92000000 52.276695746879064
92100000 56.39112250865562
92200000 53.59951334367004
92300000 53.87513309384786
92400000 53.93938405821031
92500000 56.29755136313099
92600000 53.30064396970384
92700000 53.4867059394463
92800000 48.56228599882356
92900000 48.513497025709235
93000000 32.80566361567232
93100000 35.28507067177043
93200000 47.87557375425752
93300000 48.76720879741033
93400000 22.81370918176126
93500000 39.9933682368

119300000 47.40451859284183
119400000 48.35306677909316
119500000 44.333738786503744
119600000 20.477118460271615
119700000 44.339048314283446
119800000 12.95955388725041
119900000 44.75990306843763
120000000 24.807106398302206
120100000 46.01364009211351
120200000 49.036103829794925
120300000 46.40107420339327
120400000 49.01665505103505
120500000 45.716815690701736
120600000 47.44440357106036
120700000 49.78308091319744
120800000 51.76725972909139
120900000 55.912953530978925
121000000 58.2637472469934
121100000 55.46101822394543
121200000 51.99159726593608
121300000 52.313941008273105
121400000 51.46336941517698
121500000 51.54622926636363
121600000 52.080756265280414
121700000 47.931922509072805
121800000 23.05641512140737
121900000 48.337970173617016
122000000 13.477199242243893
122100000 37.95032131054943
122200000 30.826457655312836
122300000 51.49068152211951
122400000 48.339086030704586
122500000 53.62458620244102
122600000 50.19543369078284
122700000 54.03998011435252
1228000

...

638600000 1.2937688906903444
638700000 1.239147368211176
638800000 0.8104953497938288
638900000 1.1669019745850921
639000000 1.3510154685694182
639100000 1.9476804258357454
639200000 1.9593905435233891
639300000 2.074119825621383
best so far: 0
type: [1, 1, 2, 13, 5] 130
cases of this type: 5492500
639400000 2.020878400874072
639500000 1.4705934399307865
639600000 1.8111636195864378
639700000 1.0737777606086167
639800000 1.0624504110230506
639900000 0.9061592827300993
640000000 0.9462206789145767
640100000 1.078969297338438
640200000 1.5988034454870577
640300000 1.9167963300102202
640400000 1.713047792382614
640500000 2.274475914853769
640600000 2.060566755796131
640700000 2.2903099200289128
640800000 1.3114374053713778
640900000 1.0388446731943044
641000000 0.9311338407872399
641100000 1.0628689328804004
641200000 1.699405153567036
641300000 1.597246302245291
641400000 1.8625909187847065
641500000 1.8076130081387705
641600000 2.5208142778408047
641700000 1.8468459477601409
641800000 1

665300000 1.373111531840259
665400000 0.3800157042030242
665500000 0.3418592018573787
665600000 0.3380316870169739
665700000 0.3376126670674331
665800000 0.336970219615876
665900000 0.3360261223092854
666000000 0.3345135691374434
666100000 0.33497807444516425
666200000 0.34580067871997555
666300000 0.3359081129554755
666400000 1.6043285552706221
666500000 2.7683429875409207
666600000 2.013260432772059
666700000 1.6602068726520682
666800000 1.839698435409504
666900000 1.5105978583013087
667000000 1.4718326341197578
667100000 1.764473789783007
667200000 2.458002004804322
best so far: 0
type: [1, 1, 10, 13, 1] 130
cases of this type: 219700
667300000 0.9108091002962205
667400000 0.5112211182112495
best so far: 0
type: [1, 1, 13, 1, 10] 130
cases of this type: 1690000
667500000 1.6623870149148487
667600000 1.933030416654855
667700000 1.7440525084131597
667800000 2.0359672997872673
667900000 2.436102162227023
668000000 1.0766739348376173
668100000 0.34238128842049903
668200000 0.34253133737

691100000 1.4143787908865153
691200000 0.498664241438663
691300000 1.3706395500438366
691400000 1.4518645144334654
691500000 1.4548924133244308
691600000 1.3218507253890648
691700000 1.6907158243283011
691800000 0.8497450926246113
691900000 1.2491935751255188
692000000 2.3593213969827773
692100000 1.9995738754715409
692200000 1.4096323114467446
692300000 0.759330850381885
692400000 1.0766064229775958
692500000 1.3443870087760375
692600000 1.9956680754526277
692700000 1.530748815826922
692800000 0.9051795694700142
692900000 0.6485663918935061
693000000 1.3112903222647667
693100000 1.32718835076585
693200000 1.2698643930507978
693300000 1.031248770855528
693400000 0.9575306168862012
693500000 1.723810649337363
693600000 1.7442290800528288
693700000 1.8185481186333559
693800000 1.2588590330467344
693900000 1.2746628127690156
694000000 0.7387409597467715
694100000 1.1865180404940188
694200000 1.5056747768770398
694300000 1.8518059222780978
694400000 1.0179135925184946
694500000 0.999682619

718400000 0.3643497615678529
718500000 0.3108033982278102
718600000 0.3275455212230318
best so far: 0
type: [1, 2, 5, 13, 1] 130
cases of this type: 109850
718700000 0.3285531974516987
best so far: 0
type: [1, 2, 13, 1, 5] 130
cases of this type: 211250
718800000 0.10573339701365365
718900000 0.11889437084035741
719000000 0.12090621981081962
best so far: 0
type: [1, 2, 13, 5, 1] 130
cases of this type: 42250
best so far: 0
type: [1, 2, 65, 1, 1] 130
cases of this type: 8450
best so far: 0
type: [1, 5, 1, 1, 26] 130
cases of this type: 2284880
719100000 0.2379910608527548
719200000 0.4124480633695126
719300000 0.29725640752532484
719400000 0.265378034482587
719500000 0.2888069367605905
719600000 0.2668987337312957
719700000 0.2805276440564275
719800000 0.26425477877425085
719900000 0.26321765107909445
720000000 0.30846974069212474
720100000 0.1694596649910523
720200000 0.11445990986087919
720300000 0.1136454484053115
720400000 0.11348534692931307
720500000 0.22931472554767132
720600000 

738500000 0.1444815206828197
738600000 0.17670168833768832
738700000 0.14291301369330286
738800000 0.16302431158522365
738900000 0.18202886195344462
739000000 0.15165162017429537
739100000 0.17006870081923012
739200000 0.17910927065864732
739300000 0.15665886613075733
739400000 0.1729812053226789
739500000 0.16883133188234434
739600000 0.1594411237415751
739700000 0.14267713980862498
739800000 0.1376400790608234
739900000 0.14955800985340142
740000000 0.1970198831413448
740100000 0.13650224582260448
740200000 0.1456562408928315
740300000 0.15596293901467523
740400000 0.1402773623690029
740500000 0.13430076888784037
740600000 0.16592245823197366
740700000 0.1397564002207948
740800000 0.1334403607601477
740900000 0.14112800530501207
741000000 0.16190431879289216
741100000 0.11615604664436843
741200000 0.1244506731251081
741300000 0.11824690317690174
741400000 0.24536043955063025
741500000 0.1225826711309433
741600000 0.12361762347667482
741700000 0.16052920358305245
741800000 0.126927873